##### **** These pip install need to be adapted to use the appropriate release level. Currently used for testing alpha release of transforms 1.0

In [38]:
%%capture
!pip install 'data-prep-toolkit-transforms[language]==1.0.0a0'
import pyarrow.parquet as pq
import pandas as pd

configur and run web2parquet

In [39]:
# Must enable nested asynchronous io in a notebook as the crawler uses coroutine to speed up acquisition and downloads
import nest_asyncio
nest_asyncio.apply()

In [40]:
%%capture
from dpk_web2parquet.transform import Web2Parquet
Web2Parquet(urls= ['https://arxiv.org/pdf/2408.09869'],
                    depth=2, 
                    downloads=10,
                    folder='downloads').transform()


In [25]:
##### **** The specified downloads folder will include the downloaded file(s).
#import glob
#glob.glob("downloads/*") 

Configure and run Pdf2Parquet

In [ ]:
from dpk_pdf2parquet.transform_python import Pdf2Parquet
Pdf2Parquet(input_folder= "downloads", 
               output_folder= "pdf2parquet-files", 
               data_files_to_use=['.pdf'],
               pdf2parquet_contents_type='text/markdown').transform()

In [29]:
##### **** To explote the output from pdf2parquet, run the code below
#table = pq.read_table('pdf2parquet-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and Run DocChunk

In [ ]:
%%capture
from dpk_doc_chunk.transform_python import DocChunk
DocChunk(input_folder='pdf2parquet-files',
        output_folder='doc-chunk-files',
        doc_chunk_chunking_type= "li_markdown").transform()

In [31]:
##### **** To explote the output from doc-chunk, run the code below
#table = pq.read_table('doc-chunk-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and Run Exact dedup

In [ ]:
from dpk_ededup.transform_python import Ededup
Ededup(input_folder="doc-chunk-files",
    output_folder="dedup-files",
    ededup_doc_column="contents",
    ededup_doc_id_column="document_id").transform()

In [33]:
##### **** To explote the output from eDedup, run the code below
#table = pq.read_table('dedup-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and run Land Id

In [ ]:
from dpk_lang_id.transform_python import LangId
LangId(input_folder= "dedup-files",
        output_folder= "langId-files",
        lang_id_model_credential= "PUT YOUR OWN HUGGINGFACE CREDENTIAL",
        lang_id_model_kind= "fasttext",
        lang_id_model_url= "facebook/fasttext-language-identification",
        lang_id_content_column_name= "contents").transform()

In [35]:
##### **** To explote the output from langId, run the code below
#table = pq.read_table('langId-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and run Doc Quality

In [ ]:
%%capture
from dpk_doc_quality.transform_python import DocQuality
DocQuality(input_folder='dedup-files',
            output_folder= 'doc-quality-files',
            docq_text_lang = "en",
            docq_doc_content_column ="contents").transform()

In [37]:
##### **** To explote the output from Doc Quality, run the code below
#table = pq.read_table('doc-quality-files/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()